### Pyodbc 연결하여 sql 불러오기

In [1]:
import os, pandas as pd, numpy as np
import pyodbc, datetime
from datetime import timedelta, datetime

In [2]:
## 디렉토리 확인
os.getcwd()

'C:\\Users\\SORA\\Desktop\\MIMIC4_PRBC\\ipynb_code'

In [3]:
## pyodbc 연결 setting
IPadr = 'DESKTOP-OFTEOMM'
Database_n = 'analysis_db'
Database_m = 'MIMIC4'

In [4]:
## analysis_db connect
q = pyodbc.connect(DRIVER='{SQL Server}', SERVER=IPadr, DATABASE=Database_n)

In [5]:
## MIMIC4_db connect
m = pyodbc.connect(DRIVER='{SQL Server}', SERVER=IPadr, DATABASE=Database_m)

In [6]:
## 각 table 쿼리에 입력
query_1 = """select * from MIMIC4_sysBP_first"""
query_2 = """select * from MIMIC4_meanBP_first"""
query_3 = """select * from MIMIC4_diaBP_first"""
query_4 = """select * from MIMIC4_HR_first"""
query_5 = """select * from MIMIC4_RR_first"""
query_6 = """select * from MIMIC4_BT_first"""
query_7 = """select * from MIMIC4_spo2_first"""
query_8 = """select * from PRBC_icu_24h_distinct_subject_id"""
query_9 = """select * from PRBC_icu_24hr"""
query_10 = """select * from PRBC_icu_12hr"""
query_11 = """select * from icu_addmissions"""
query_12 = """select * from icu_paients"""
query_13 = """select * from MIMIC4_control"""

In [7]:
## 쿼리 불러오기
PRBC_id_list_24h = pd.read_sql(query_8, q)
PRBC_24h = pd.read_sql(query_9, q)
PRBC_12h = pd.read_sql(query_10, q)
icu_adm = pd.read_sql(query_11, q)
icu_pat = pd.read_sql(query_12, q)

In [8]:
## 쿼리 불러오기
sysBP = pd.read_sql(query_1, q)
meanBP = pd.read_sql(query_2, q)
diaBP = pd.read_sql(query_3, q)
HR = pd.read_sql(query_4, q)
RR = pd.read_sql(query_5, q)
BT = pd.read_sql(query_6, q)
spo2 = pd.read_sql(query_7, q)

In [9]:
## object to datetime
PRBC_24h['intime'] = pd.to_datetime(PRBC_24h['intime'])
PRBC_24h['outtime'] = pd.to_datetime(PRBC_24h['outtime'])
PRBC_24h['starttime'] = pd.to_datetime(PRBC_24h['starttime'])
PRBC_24h['endtime'] = pd.to_datetime(PRBC_24h['endtime'])

In [10]:
## subject_id 기준으로 정렬하여 중복 제거
case_list = PRBC_24h.sort_values(['subject_id', 'intime']).drop_duplicates(['subject_id'],keep='first').reset_index(drop=True)

In [11]:
mimic_control = pd.read_sql(query_13, q)

In [12]:
mimic_control['intime'] = pd.to_datetime(mimic_control['intime'])
mimic_control['outtime'] = pd.to_datetime(mimic_control['outtime'])

In [13]:
## PRBC 제외 control, 동일하게 subject_id 기준으로 정렬하여 중복 제거
control_list = mimic_control.sort_values(['subject_id', 'intime']).drop_duplicates(['subject_id'],keep='first').reset_index(drop=True)

### vital sign merge

In [14]:
sysBP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69580 entries, 0 to 69579
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   subject_id    69580 non-null  int64  
 1   hadm_id       69580 non-null  int64  
 2   stay_id       69580 non-null  int64  
 3   charttime     69580 non-null  object 
 4   storetime     69574 non-null  object 
 5   itemid        69580 non-null  int64  
 6   value         69580 non-null  object 
 7   valuenum      69580 non-null  float64
 8   valueuom      69580 non-null  object 
 9   warning       69580 non-null  int64  
 10  charttime_RN  69580 non-null  int64  
dtypes: float64(1), int64(6), object(4)
memory usage: 5.8+ MB


In [15]:
## id, charttime, 측정치 변수만 불러서 정렬
sysBP = sysBP[['subject_id','hadm_id','stay_id','charttime','valuenum']].sort_values(['subject_id','stay_id','charttime'])
meanBP = meanBP[['subject_id','hadm_id','stay_id','charttime','valuenum']].sort_values(['subject_id','stay_id','charttime'])
diaBP = diaBP[['subject_id','hadm_id','stay_id','charttime','valuenum']].sort_values(['subject_id','stay_id','charttime'])
HR = HR[['subject_id','hadm_id','stay_id','charttime','valuenum']].sort_values(['subject_id','stay_id','charttime'])
RR = RR[['subject_id','hadm_id','stay_id','charttime','valuenum']].sort_values(['subject_id','stay_id','charttime'])
spo2 = spo2[['subject_id','hadm_id','stay_id','charttime','valuenum']].sort_values(['subject_id','stay_id','charttime'])
BT = BT[['subject_id','hadm_id','stay_id','charttime','bt']].sort_values(['subject_id','stay_id','charttime'])

In [16]:
## 컬럼명 변경, BT는 화씨-> 섭씨로 단위 변경시 변수명 이미 통합
sysBP.rename(columns = {'valuenum' : 'sys_bp'}, inplace = True)
meanBP.rename(columns = {'valuenum' : 'mean_bp'}, inplace = True)
diaBP.rename(columns = {'valuenum' : 'dia_bp'}, inplace = True)
HR.rename(columns = {'valuenum' : 'HR'}, inplace = True)
RR.rename(columns = {'valuenum' : 'RR'}, inplace = True)
spo2.rename(columns = {'valuenum' : 'spo2'}, inplace = True)

In [17]:
vital=pd.merge(sysBP, meanBP, how='outer', on=['subject_id','hadm_id','stay_id','charttime'])

In [18]:
vital=pd.merge(vital, diaBP, how='outer', on=['subject_id','hadm_id','stay_id','charttime'])
vital=pd.merge(vital, HR, how='outer', on=['subject_id','hadm_id','stay_id','charttime'])
vital=pd.merge(vital, RR, how='outer', on=['subject_id','hadm_id','stay_id','charttime'])
vital=pd.merge(vital, BT, how='outer', on=['subject_id','hadm_id','stay_id','charttime'])
vital=pd.merge(vital, spo2, how='outer', on=['subject_id','hadm_id','stay_id','charttime'])

In [19]:
vital = vital.sort_values(['subject_id','charttime'])

In [20]:
vital['charttime'] = pd.to_datetime(vital['charttime'])

* case group

In [21]:
case_df=case_list[['subject_id','hadm_id','stay_id','intime','outtime']]

In [22]:
case_vital=pd.merge(case_df, vital, how='left', on=['subject_id','hadm_id','stay_id'])

In [23]:
case_vital = case_vital[case_vital['intime']<=case_vital['charttime']]
case_vital = case_vital[case_vital['charttime']<=case_vital['outtime']]

In [24]:
case_vital['timedelta'] = case_vital['charttime'] - case_vital['intime']
case_vital['timedelta_min'] = (case_vital['charttime'] - case_vital['intime'])/pd.Timedelta('1 minutes')

In [25]:
## intime과 charttime사이에 평균이 161분....;;;
sum(case_vital['timedelta_min'])/len(case_vital['timedelta_min'])

161.51131449819667

In [26]:
## 중환자실 입실 이후 60분 이내 측정된 초기값만...
case_vital = case_vital[case_vital['timedelta_min']<=60]

In [27]:
## groupby 위한 id 열 복제
case_vital['subject_id_index'] = case_vital['subject_id']

In [28]:
case_vital_groupby=case_vital.groupby('subject_id_index').fillna(method='ffill').fillna(method='bfill')

In [29]:
case_vital_df = case_vital_groupby.sort_values(['subject_id','intime']).drop_duplicates(['subject_id'], keep='first')

In [30]:
case_vital_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6080 entries, 0 to 29272
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype          
---  ------         --------------  -----          
 0   subject_id     6080 non-null   int64          
 1   hadm_id        6080 non-null   int64          
 2   stay_id        6080 non-null   int64          
 3   intime         6080 non-null   datetime64[ns] 
 4   outtime        6080 non-null   datetime64[ns] 
 5   charttime      6080 non-null   datetime64[ns] 
 6   sys_bp         6080 non-null   float64        
 7   mean_bp        6080 non-null   float64        
 8   dia_bp         6080 non-null   float64        
 9   HR             6080 non-null   float64        
 10  RR             6080 non-null   float64        
 11  bt             6080 non-null   float64        
 12  spo2           6080 non-null   float64        
 13  timedelta      6080 non-null   timedelta64[ns]
 14  timedelta_min  6080 non-null   float64        
dtypes: 

In [31]:
case_vital_df.describe()

,subject_id,hadm_id,stay_id,sys_bp,mean_bp,dia_bp,HR,RR,bt,spo2,timedelta,timedelta_min
count,6.080000e+03,6.080000e+03,6.080000e+03,6080.000000,6080.000000,6080.000000,6080.000000,6080.000000,6080.000000,6080.000000,6080,6080.000000
mean,1.502008e+07,2.499067e+07,3.494391e+07,118.939309,78.502303,64.200822,92.626645,19.720724,36.715607,97.190921,0 days 00:12:51.608552631,12.860143
std,2.885042e+06,2.908410e+06,2.882673e+06,24.605312,19.802911,18.064077,20.323970,6.095659,6.619095,11.964447,0 days 00:13:41.623291872,13.693722
min,1.000216e+07,2.000342e+07,3.000097e+07,9.000000,-9.000000,1.000000,0.000000,0.000000,-17.777778,9.000000,0 days 00:00:00,0.000000
25%,1.252798e+07,2.241777e+07,3.238525e+07,102.000000,66.000000,52.000000,78.000000,15.000000,36.388889,96.000000,0 days 00:02:52,2.866667
50%,1.506340e+07,2.498487e+07,3.494527e+07,117.000000,76.000000,62.000000,91.000000,19.000000,36.722222,98.000000,0 days 00:08:00,8.000000
75%,1.749170e+07,2.753421e+07,3.742599e+07,135.000000,89.000000,74.000000,105.000000,23.000000,37.055556,100.000000,0 days 00:17:33.250000,17.554167
max,1.999678e+07,2.999661e+07,3.999915e+07,233.000000,348.000000,192.000000,188.000000,51.000000,537.166667,963.000000,0 days 01:00:00,60.000000


In [32]:
## 99백분위수 극단값 확인
print(case_vital_df['sys_bp'].quantile(0.99))
print(case_vital_df['dia_bp'].quantile(0.99))
print(case_vital_df['mean_bp'].quantile(0.99))
print(case_vital_df['HR'].quantile(0.99))
print(case_vital_df['RR'].quantile(0.99))
print(case_vital_df['bt'].quantile(0.99))
print(case_vital_df['spo2'].quantile(0.99))

183.0
117.0
133.0
145.0
38.0
38.94444444444444
100.0


In [33]:
## 이상치 99백분위수 극단값으로 대체
case_vital_df['sys_bp']=case_vital_df['sys_bp'].apply(lambda x:x if x>=0 and x<=183.0 else 183.0)
case_vital_df['dia_bp']=case_vital_df['dia_bp'].apply(lambda x:x if x>=0 and x<=117.0 else 117.0)
case_vital_df['mean_bp']=case_vital_df['mean_bp'].apply(lambda x:x if x>=0 and x<=133.0 else 133.0)
case_vital_df['HR']=case_vital_df['HR'].apply(lambda x:x if x>=0 and x<=145.0 else 145.0)
case_vital_df['RR']=case_vital_df['RR'].apply(lambda x:x if x>=0 and x<=38.0 else 38.0)
case_vital_df['bt']=case_vital_df['bt'].apply(lambda x:x if x>=0 and x<=38.9 else 38.9)
case_vital_df['spo2']=case_vital_df['spo2'].apply(lambda x:x if x>=0 and x<=100 else 100.0)

In [34]:
## CSV내보내기
case_vital_df.to_csv('./case_vital.csv', header=True, index=False)

* control_group

In [35]:
control_df=control_list[['subject_id','hadm_id','stay_id','intime','outtime']]

In [36]:
control_vital=pd.merge(control_df, vital, how='left', on=['subject_id','hadm_id','stay_id'])

In [37]:
control_vital = control_vital[control_vital['intime']<=control_vital['charttime']]
control_vital = control_vital[control_vital['charttime']<=control_vital['outtime']]

In [38]:
control_vital['timedelta'] = control_vital['charttime'] - control_vital['intime']
control_vital['timedelta_min'] = (control_vital['charttime'] - control_vital['intime'])/pd.Timedelta('1 minutes')

In [39]:
## intime과 charttime사이에 평균이 86분....;;;
sum(control_vital['timedelta_min'])/len(control_vital['timedelta_min'])

86.84417035140882

In [40]:
## 중환자실 입실 이후 60분 이내 측정된 초기값만...
control_vital = control_vital[control_vital['timedelta_min']<=60]

In [41]:
## groupby 위한 id 열 복제
control_vital['subject_id_index'] = control_vital['subject_id']

In [42]:
control_vital_groupby=control_vital.groupby('subject_id_index').fillna(method='ffill').fillna(method='bfill')

In [43]:
control_vital_df = control_vital_groupby.sort_values(['subject_id','intime']).drop_duplicates(['subject_id'], keep='first')

In [44]:
control_vital_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29805 entries, 0 to 133251
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype          
---  ------         --------------  -----          
 0   subject_id     29805 non-null  int64          
 1   hadm_id        29805 non-null  int64          
 2   stay_id        29805 non-null  int64          
 3   intime         29805 non-null  datetime64[ns] 
 4   outtime        29805 non-null  datetime64[ns] 
 5   charttime      29805 non-null  datetime64[ns] 
 6   sys_bp         29805 non-null  float64        
 7   mean_bp        29805 non-null  float64        
 8   dia_bp         29805 non-null  float64        
 9   HR             29805 non-null  float64        
 10  RR             29805 non-null  float64        
 11  bt             29805 non-null  float64        
 12  spo2           29805 non-null  float64        
 13  timedelta      29805 non-null  timedelta64[ns]
 14  timedelta_min  29805 non-null  float64        
dtypes

In [45]:
control_vital_df.describe()

,subject_id,hadm_id,stay_id,sys_bp,mean_bp,dia_bp,HR,RR,bt,spo2,timedelta,timedelta_min
count,2.980500e+04,2.980500e+04,2.980500e+04,29805.000000,29805.000000,29805.000000,29805.000000,29805.000000,29805.000000,29805.000000,29805,29805.000000
mean,1.499664e+07,2.496541e+07,3.501378e+07,129.045529,91.429123,80.842979,88.270391,19.430599,36.729900,96.690656,0 days 00:13:22.037678241,13.367295
std,2.893111e+06,2.883332e+06,2.874572e+06,74.455549,812.090588,846.740098,20.946138,6.181594,1.791594,4.122779,0 days 00:13:44.662252307,13.744371
min,1.000003e+07,2.000009e+07,3.000136e+07,0.000000,-34.000000,0.000000,0.000000,0.000000,-17.777778,0.000000,0 days 00:00:00,0.000000
25%,1.247845e+07,2.248068e+07,3.254898e+07,111.000000,74.000000,60.000000,74.000000,15.000000,36.444444,95.000000,0 days 00:03:00,3.000000
50%,1.501437e+07,2.494102e+07,3.501352e+07,128.000000,85.000000,70.000000,86.000000,19.000000,36.777778,98.000000,0 days 00:09:00,9.000000
75%,1.752024e+07,2.745441e+07,3.750486e+07,144.000000,98.000000,82.000000,101.000000,23.000000,37.100000,100.000000,0 days 00:18:01,18.016667
max,1.999999e+07,2.999983e+07,3.999871e+07,12262.000000,140119.000000,100105.000000,941.000000,152.000000,99.200000,194.000000,0 days 01:00:00,60.000000


In [46]:
## 99백분위수 극단값 확인
print(control_vital_df['sys_bp'].quantile(0.99))
print(control_vital_df['dia_bp'].quantile(0.99))
print(control_vital_df['mean_bp'].quantile(0.99))
print(control_vital_df['HR'].quantile(0.99))
print(control_vital_df['RR'].quantile(0.99))
print(control_vital_df['bt'].quantile(0.99))
print(control_vital_df['spo2'].quantile(0.99))

193.0
124.0
137.0
143.0
38.0
39.0
100.0


In [47]:
## 이상치 99백분위수 극단값으로 대체
control_vital_df['sys_bp']=control_vital_df['sys_bp'].apply(lambda x:x if x>=0 and x<=193.0 else 193.0)
control_vital_df['dia_bp']=control_vital_df['dia_bp'].apply(lambda x:x if x>=0 and x<=124.0 else 124.0)
control_vital_df['mean_bp']=control_vital_df['mean_bp'].apply(lambda x:x if x>=0 and x<=137.0 else 137.0)
control_vital_df['HR']=control_vital_df['HR'].apply(lambda x:x if x>=0 and x<=143.0 else 143.0)
control_vital_df['RR']=control_vital_df['RR'].apply(lambda x:x if x>=0 and x<=38.0 else 38.0)
control_vital_df['bt']=control_vital_df['bt'].apply(lambda x:x if x>=0 and x<=39.0 else 39.0)
control_vital_df['spo2']=control_vital_df['spo2'].apply(lambda x:x if x>=0 and x<=100 else 100.0)

In [48]:
## CSV내보내기
control_vital_df.to_csv('./control_vital.csv', header=True, index=False)

## ICU social demographic

* case group

In [49]:
case_val = case_list[['subject_id','hadm_id','stay_id','first_careunit','intime','outtime','los']]

In [50]:
case_val_adm = pd.merge(case_val, icu_adm, how='left', on=['subject_id'])

In [51]:
case_val_adm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44110 entries, 0 to 44109
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   subject_id      44110 non-null  int64         
 1   hadm_id         44110 non-null  int64         
 2   stay_id         44110 non-null  int64         
 3   first_careunit  44110 non-null  object        
 4   intime          44110 non-null  datetime64[ns]
 5   outtime         44110 non-null  datetime64[ns]
 6   los             44110 non-null  float64       
 7   admittime       44110 non-null  object        
 8   dischtime       44110 non-null  object        
 9   admission_type  44110 non-null  object        
 10  insurance       44110 non-null  object        
 11  language        44110 non-null  object        
 12  marital_status  43299 non-null  object        
 13  ethnicity       44110 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(3), object(8)


In [52]:
case_val_adm['admittime'] = pd.to_datetime(case_val_adm['admittime'])
case_val_adm['dischtime'] = pd.to_datetime(case_val_adm['dischtime'])

In [53]:
case_val_adm = case_val_adm[case_val_adm['admittime']<=case_val_adm['intime']]
case_val_adm = case_val_adm[case_val_adm['intime']<=case_val_adm['dischtime']]

In [54]:
case_demo = pd.merge(case_val_adm, icu_pat, how='left', on=['subject_id'])

In [55]:
case_demo

,subject_id,hadm_id,stay_id,first_careunit,intime,outtime,los,admittime,dischtime,admission_type,insurance,language,marital_status,ethnicity,gender,anchor_age,anchor_year,anchor_year_group
0,10002155,28994087,37365187,Medical/Surgical Intensive Care Unit (MICU/SICU),2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447,2130-09-23 21:59:00,2130-09-29 18:55:00,EW EMER.,Other,ENGLISH,MARRIED,WHITE,F,80,2128,2008 - 2010
1,10002428,23473524,39060160,Surgical Intensive Care Unit (SICU),2156-05-11 14:52:30,2156-05-22 14:16:46,10.975185,2156-05-11 14:49:00,2156-05-22 14:16:00,EW EMER.,Medicare,ENGLISH,WIDOWED,WHITE,F,80,2155,2011 - 2013
2,10003400,20214994,38355205,PACU,2137-02-25 19:43:32,2137-03-10 21:29:36,13.073657,2137-02-24 10:00:00,2137-03-19 15:45:00,URGENT,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,F,72,2134,2011 - 2013
3,10004401,29988601,37470986,Medical Intensive Care Unit (MICU),2144-01-26 22:28:04,2144-02-06 13:44:15,10.636238,2144-01-23 07:58:00,2144-02-06 11:45:00,EW EMER.,Medicare,ENGLISH,MARRIED,WHITE,M,82,2141,2008 - 2010
4,10004422,21255400,39826382,Cardiac Vascular Intensive Care Unit (CVICU),2111-01-17 09:44:50,2111-01-23 18:18:46,6.356898,2111-01-15 14:55:00,2111-01-25 15:00:00,EW EMER.,Medicare,ENGLISH,WIDOWED,WHITE,M,78,2111,2011 - 2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8852,19992312,25379817,35622208,Surgical Intensive Care Unit (SICU),2158-06-28 06:00:00,2158-06-29 16:53:59,1.454155,2158-06-28 04:44:00,2158-07-01 15:52:00,EW EMER.,Medicare,ENGLISH,SINGLE,WHITE,M,86,2158,2008 - 2010
8853,19993726,20161163,32385354,Surgical Intensive Care Unit (SICU),2166-09-14 17:17:40,2166-09-23 18:55:58,9.068264,2166-09-14 07:15:00,2166-10-02 17:02:00,SURGICAL SAME DAY ADMISSION,Other,ENGLISH,MARRIED,WHITE,F,52,2166,2008 - 2010
8854,19994379,27334101,35953321,Neuro Surgical Intensive Care Unit (Neuro SICU),2131-06-06 06:35:18,2131-06-07 22:43:21,1.672257,2131-05-30 16:54:00,2131-06-21 17:20:00,OBSERVATION ADMIT,Medicare,ENGLISH,MARRIED,WHITE,M,68,2128,2014 - 2016
8855,19995127,27369164,30593945,Medical/Surgical Intensive Care Unit (MICU/SICU),2138-06-07 07:15:05,2138-06-09 17:44:46,2.437280,2138-06-06 18:00:00,2138-06-12 01:48:00,EW EMER.,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,M,78,2133,2008 - 2010


In [145]:
case_demo[['ethnicity']].value_counts()

ethnicity                    
WHITE                            6053
BLACK/AFRICAN AMERICAN            853
UNKNOWN                           737
OTHER                             391
ASIAN                             343
HISPANIC/LATINO                   328
UNABLE TO OBTAIN                  130
AMERICAN INDIAN/ALASKA NATIVE      22
dtype: int64

In [57]:
case_demo_df = case_demo.copy()

In [58]:
##중환자실 입실 연도만 추출 (나이 계산 다시하기 위해)
case_demo_df['intime_year']=[str(case_demo_df['intime'][i]) for i in range(len(case_demo_df))]

In [59]:
case_demo_df['intime_year']=case_demo_df['intime_year'].str.slice(start=0, stop=4).astype('int')

In [60]:
case_demo_df['diff']=case_demo_df['intime_year']-case_demo_df['anchor_year']

In [61]:
case_demo_df['age']=case_demo_df['anchor_age']+case_demo_df['diff']

In [62]:
##성별 변환 M=1, F=0
case_demo_df['gender']=case_demo_df['gender'].apply(lambda x: 1 if x == 'M' else 0)

In [63]:
##language english=1, ?=0
case_demo_df['language']=case_demo_df['language'].apply(lambda x: 1 if x == 'ENGLISH' else 0)

In [64]:
obs = ["EU OBSERVATION",'OBSERVATION ADMIT','EU OBSERVATION','DIRECT OBSERVATION','AMBULATORY OBSERVATION']

In [65]:
emer = ['DIRECT EMER.', 'EW EMER.', 'URGENT']

In [66]:
def admission_category(x):
    for i in range(0,len(x)):
        if x[i] in obs:
            x[i] = 'OBSER'
        elif x[i] in emer:
            x[i] = 'EMER'
        elif x[i] == 'SURGICAL SAME DAY ADMISSION':
            x[i] = 'SURGICAL'
        else:
            x[i]

In [67]:
admission_category(case_demo_df['admission_type'])

<ipython-input-66-551e530b9a19>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = 'EMER'
<ipython-input-66-551e530b9a19>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = 'OBSER'
<ipython-input-66-551e530b9a19>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = 'SURGICAL'


In [68]:
def careunit_category(x):
    for i in range(0,len(x)):
        if x[i] == 'Medical Intensive Care Unit (MICU)':
            x[i] = 'MICU'
        elif x[i] =='Cardiac Vascular Intensive Care Unit (CVICU)':
            x[i] = 'CVICU'
        elif x[i] == 'Medical/Surgical Intensive Care Unit (MICU/SICU)':
            x[i] = 'M/SICU'
        elif x[i] == 'Trauma SICU (TSICU)':
            x[i] = 'TSICU'
        elif x[i] =='Surgical Intensive Care Unit (SICU)':
            x[i] = 'SICU'
        elif x[i] =='Coronary Care Unit (CCU)':
            x[i] = 'CCU'
        else:
            x[i] = 'Other'

In [69]:
careunit_category(case_demo_df['first_careunit'])

<ipython-input-68-1a177f3cd61e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = 'M/SICU'
<ipython-input-68-1a177f3cd61e>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = 'SICU'
<ipython-input-68-1a177f3cd61e>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = 'Other'
<ipython-input-68-1a177f3cd61e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the

In [70]:
def ethnic_category(x):
    for i in range(0,len(x)):
        if x[i] == 'UNABLE TO OBTAIN':
            x[i] = 'UNKNOWN'
        elif x[i] == 'AMERICAN INDIAN/ALASKA NATIVE':
            x[i] = 'OTHER'
        else:
            x[i]

In [71]:
ethnic_category(case_demo_df['ethnicity'])

<ipython-input-70-7736dcc09ced>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = 'UNKNOWN'
<ipython-input-70-7736dcc09ced>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = 'OTHER'


In [72]:
case_demo_df[['ethnicity']].value_counts()

ethnicity             
WHITE                     6053
UNKNOWN                    867
BLACK/AFRICAN AMERICAN     853
OTHER                      413
ASIAN                      343
HISPANIC/LATINO            328
dtype: int64

In [73]:
case_demo_df

,subject_id,hadm_id,stay_id,first_careunit,intime,outtime,los,admittime,dischtime,admission_type,...,language,marital_status,ethnicity,gender,anchor_age,anchor_year,anchor_year_group,intime_year,diff,age
0,10002155,28994087,37365187,M/SICU,2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447,2130-09-23 21:59:00,2130-09-29 18:55:00,EMER,...,1,MARRIED,WHITE,0,80,2128,2008 - 2010,2130,2,82
1,10002428,23473524,39060160,SICU,2156-05-11 14:52:30,2156-05-22 14:16:46,10.975185,2156-05-11 14:49:00,2156-05-22 14:16:00,EMER,...,1,WIDOWED,WHITE,0,80,2155,2011 - 2013,2156,1,81
2,10003400,20214994,38355205,Other,2137-02-25 19:43:32,2137-03-10 21:29:36,13.073657,2137-02-24 10:00:00,2137-03-19 15:45:00,EMER,...,1,MARRIED,BLACK/AFRICAN AMERICAN,0,72,2134,2011 - 2013,2137,3,75
3,10004401,29988601,37470986,MICU,2144-01-26 22:28:04,2144-02-06 13:44:15,10.636238,2144-01-23 07:58:00,2144-02-06 11:45:00,EMER,...,1,MARRIED,WHITE,1,82,2141,2008 - 2010,2144,3,85
4,10004422,21255400,39826382,CVICU,2111-01-17 09:44:50,2111-01-23 18:18:46,6.356898,2111-01-15 14:55:00,2111-01-25 15:00:00,EMER,...,1,WIDOWED,WHITE,1,78,2111,2011 - 2013,2111,0,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8852,19992312,25379817,35622208,SICU,2158-06-28 06:00:00,2158-06-29 16:53:59,1.454155,2158-06-28 04:44:00,2158-07-01 15:52:00,EMER,...,1,SINGLE,WHITE,1,86,2158,2008 - 2010,2158,0,86
8853,19993726,20161163,32385354,SICU,2166-09-14 17:17:40,2166-09-23 18:55:58,9.068264,2166-09-14 07:15:00,2166-10-02 17:02:00,SURGICAL,...,1,MARRIED,WHITE,0,52,2166,2008 - 2010,2166,0,52
8854,19994379,27334101,35953321,Other,2131-06-06 06:35:18,2131-06-07 22:43:21,1.672257,2131-05-30 16:54:00,2131-06-21 17:20:00,OBSER,...,1,MARRIED,WHITE,1,68,2128,2014 - 2016,2131,3,71
8855,19995127,27369164,30593945,M/SICU,2138-06-07 07:15:05,2138-06-09 17:44:46,2.437280,2138-06-06 18:00:00,2138-06-12 01:48:00,EMER,...,1,MARRIED,BLACK/AFRICAN AMERICAN,1,78,2133,2008 - 2010,2138,5,83


In [74]:
case_demo_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8857 entries, 0 to 8856
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   subject_id         8857 non-null   int64         
 1   hadm_id            8857 non-null   int64         
 2   stay_id            8857 non-null   int64         
 3   first_careunit     8857 non-null   object        
 4   intime             8857 non-null   datetime64[ns]
 5   outtime            8857 non-null   datetime64[ns]
 6   los                8857 non-null   float64       
 7   admittime          8857 non-null   datetime64[ns]
 8   dischtime          8857 non-null   datetime64[ns]
 9   admission_type     8857 non-null   object        
 10  insurance          8857 non-null   object        
 11  language           8857 non-null   int64         
 12  marital_status     8279 non-null   object        
 13  ethnicity          8857 non-null   object        
 14  gender  

In [75]:
## CSV내보내기
case_demo_df.to_csv('./case_demo_df.csv', header=True, index=False)

* control group

In [76]:
control_val = control_list[['subject_id','hadm_id','stay_id','first_careunit','intime','outtime','los']]

In [77]:
control_val_adm = pd.merge(control_val, icu_adm, how='left', on=['subject_id'])

In [78]:
control_val_adm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132177 entries, 0 to 132176
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   subject_id      132177 non-null  int64         
 1   hadm_id         132177 non-null  int64         
 2   stay_id         132177 non-null  int64         
 3   first_careunit  132177 non-null  object        
 4   intime          132177 non-null  datetime64[ns]
 5   outtime         132177 non-null  datetime64[ns]
 6   los             132177 non-null  float64       
 7   admittime       132177 non-null  object        
 8   dischtime       132177 non-null  object        
 9   admission_type  132177 non-null  object        
 10  insurance       132177 non-null  object        
 11  language        132177 non-null  object        
 12  marital_status  127866 non-null  object        
 13  ethnicity       132177 non-null  object        
dtypes: datetime64[ns](2), float64(1), in

In [79]:
control_val_adm['admittime'] = pd.to_datetime(control_val_adm['admittime'])
control_val_adm['dischtime'] = pd.to_datetime(control_val_adm['dischtime'])

In [80]:
control_val_adm = control_val_adm[control_val_adm['admittime']<=control_val_adm['intime']]
control_val_adm = control_val_adm[control_val_adm['intime']<=control_val_adm['dischtime']]

In [81]:
control_demo = pd.merge(control_val_adm, icu_pat, how='left', on=['subject_id'])

In [82]:
control_demo

,subject_id,hadm_id,stay_id,first_careunit,intime,outtime,los,admittime,dischtime,admission_type,insurance,language,marital_status,ethnicity,gender,anchor_age,anchor_year,anchor_year_group
0,10000032,29079034,37529680,Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,2180-07-23 12:35:00,2180-07-25 17:55:00,EW EMER.,Medicaid,ENGLISH,WIDOWED,WHITE,F,52,2180,2014 - 2016
1,10000980,26913865,36742749,Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535,2189-06-27 07:38:00,2189-07-03 03:00:00,EW EMER.,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,F,73,2186,2008 - 2010
2,10001217,24597018,30327812,Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032,2157-11-18 22:56:00,2157-11-25 18:00:00,EW EMER.,Other,?,MARRIED,WHITE,F,55,2157,2011 - 2013
3,10001725,25563031,37542711,Medical/Surgical Intensive Care Unit (MICU/SICU),2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588,2110-04-11 15:08:00,2110-04-14 15:00:00,EW EMER.,Other,ENGLISH,MARRIED,WHITE,F,46,2110,2011 - 2013
4,10001884,26184834,31832304,Medical Intensive Care Unit (MICU),2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817,2131-01-07 20:39:00,2131-01-20 05:15:00,OBSERVATION ADMIT,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,F,68,2122,2008 - 2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40754,19999297,21439025,35840543,Medical Intensive Care Unit (MICU),2162-08-16 05:48:32,2162-08-23 06:22:41,7.023715,2162-08-14 23:55:00,2162-08-23 04:16:00,OBSERVATION ADMIT,Other,ENGLISH,SINGLE,OTHER,M,42,2162,2008 - 2010
40755,19999442,26785317,37569254,Surgical Intensive Care Unit (SICU),2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370,2148-11-19 10:00:00,2148-12-04 16:25:00,ELECTIVE,Medicaid,ENGLISH,DIVORCED,WHITE,M,41,2146,2008 - 2010
40756,19999625,25304202,38646795,Medical/Surgical Intensive Care Unit (MICU/SICU),2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741,2139-10-10 18:06:00,2139-10-16 03:30:00,EW EMER.,Medicare,?,MARRIED,WHITE,M,81,2138,2008 - 2010
40757,19999840,21033226,39944651,Trauma SICU (TSICU),2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766,2164-09-10 13:47:00,2164-09-17 13:42:00,EW EMER.,Other,ENGLISH,WIDOWED,WHITE,M,58,2164,2008 - 2010


In [83]:
control_demo[['marital_status']].value_counts()

marital_status
MARRIED           18240
SINGLE            11177
WIDOWED            4942
DIVORCED           2838
dtype: int64

In [84]:
control_demo_df = control_demo.copy()

In [85]:
##중환자실 입실 연도만 추출 (나이 계산 다시하기 위해)
control_demo_df['intime_year']=[str(control_demo_df['intime'][i]) for i in range(len(control_demo_df))]

In [86]:
control_demo_df['intime_year']=control_demo_df['intime_year'].str.slice(start=0, stop=4).astype('int')

In [87]:
control_demo_df['diff']=control_demo_df['intime_year']-control_demo_df['anchor_year']

In [88]:
control_demo_df['age']=control_demo_df['anchor_age']+control_demo_df['diff']

In [89]:
##성별 변환 M=1, F=0
control_demo_df['gender']=control_demo_df['gender'].apply(lambda x: 1 if x == 'M' else 0)

In [90]:
##language english=1, ?=0
control_demo_df['language']=control_demo_df['language'].apply(lambda x: 1 if x == 'ENGLISH' else 0)

In [91]:
admission_category(control_demo_df['admission_type'])

<ipython-input-66-551e530b9a19>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = 'EMER'
<ipython-input-66-551e530b9a19>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = 'OBSER'
<ipython-input-66-551e530b9a19>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = 'SURGICAL'


In [92]:
careunit_category(control_demo_df['first_careunit'])

<ipython-input-68-1a177f3cd61e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = 'MICU'
<ipython-input-68-1a177f3cd61e>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = 'SICU'
<ipython-input-68-1a177f3cd61e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = 'M/SICU'
<ipython-input-68-1a177f3cd61e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the d

In [93]:
ethnic_category(control_demo_df['ethnicity'])

<ipython-input-70-7736dcc09ced>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = 'UNKNOWN'
<ipython-input-70-7736dcc09ced>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = 'OTHER'


In [94]:
control_demo_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40759 entries, 0 to 40758
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   subject_id         40759 non-null  int64         
 1   hadm_id            40759 non-null  int64         
 2   stay_id            40759 non-null  int64         
 3   first_careunit     40759 non-null  object        
 4   intime             40759 non-null  datetime64[ns]
 5   outtime            40759 non-null  datetime64[ns]
 6   los                40759 non-null  float64       
 7   admittime          40759 non-null  datetime64[ns]
 8   dischtime          40759 non-null  datetime64[ns]
 9   admission_type     40759 non-null  object        
 10  insurance          40759 non-null  object        
 11  language           40759 non-null  int64         
 12  marital_status     37197 non-null  object        
 13  ethnicity          40759 non-null  object        
 14  gender

In [95]:
control_demo_df.tail(60)

,subject_id,hadm_id,stay_id,first_careunit,intime,outtime,los,admittime,dischtime,admission_type,...,language,marital_status,ethnicity,gender,anchor_age,anchor_year,anchor_year_group,intime_year,diff,age
40699,19986341,28993079,39266345,CVICU,2169-09-07 15:56:38,2169-09-11 15:40:44,3.988958,2169-09-04 18:42:00,2169-09-17 18:10:00,EMER,...,1,MARRIED,WHITE,1,71,2169,2017 - 2019,2169,0,71
40700,19986589,25282148,39360609,CVICU,2182-08-16 10:18:47,2182-08-17 15:27:52,1.214641,2182-08-14 01:44:00,2182-08-20 13:20:00,EMER,...,1,DIVORCED,BLACK/AFRICAN AMERICAN,1,57,2182,2008 - 2010,2182,0,57
40701,19986604,21846283,35482601,M/SICU,2159-11-03 18:48:00,2159-11-04 19:56:53,1.047836,2159-11-03 17:16:00,2159-11-06 13:45:00,EMER,...,1,MARRIED,WHITE,0,71,2159,2008 - 2010,2159,0,71
40702,19986715,28333468,30775066,SICU,2150-11-29 03:59:00,2150-12-15 12:23:04,16.350046,2150-11-29 02:47:00,2151-01-14 14:20:00,EMER,...,1,SINGLE,BLACK/AFRICAN AMERICAN,0,20,2150,2014 - 2016,2150,0,20
40703,19986880,22950392,39155371,CCU,2185-07-08 01:08:06,2185-07-14 17:19:57,6.674896,2185-07-08 01:06:00,2185-07-17 15:08:00,EMER,...,1,WIDOWED,UNKNOWN,0,78,2185,2011 - 2013,2185,0,78
40704,19987071,22919294,30887540,MICU,2113-12-27 20:31:00,2113-12-28 17:12:30,0.862153,2113-12-27 19:35:00,2113-12-31 16:45:00,EMER,...,1,SINGLE,WHITE,0,91,2113,2008 - 2010,2113,0,91
40705,19987238,29437526,31846167,CCU,2164-12-17 20:56:45,2164-12-18 13:32:24,0.691424,2164-12-17 15:18:00,2164-12-19 17:50:00,EMER,...,1,SINGLE,WHITE,0,62,2163,2008 - 2010,2164,1,63
40706,19987702,26568899,37540099,Other,2131-01-15 22:29:48,2131-01-17 21:26:29,1.956030,2131-01-13 16:45:00,2131-01-23 15:00:00,OBSER,...,1,MARRIED,WHITE,1,51,2131,2017 - 2019,2131,0,51
40707,19988077,28414691,32132065,TSICU,2140-05-08 23:11:00,2140-05-13 14:32:43,4.640081,2140-05-02 20:55:00,2140-05-22 17:48:00,EMER,...,1,SINGLE,WHITE,1,53,2140,2014 - 2016,2140,0,53
40708,19989126,26509711,39497077,SICU,2146-07-07 17:09:24,2146-07-08 18:46:05,1.067141,2146-07-07 16:28:00,2146-07-15 14:30:00,EMER,...,1,MARRIED,ASIAN,0,45,2146,2008 - 2010,2146,0,45


In [96]:
control_demo[['marital_status']].value_counts()

marital_status
MARRIED           18240
SINGLE            11177
WIDOWED            4942
DIVORCED           2838
dtype: int64

In [97]:
## CSV내보내기
control_demo_df.to_csv('./control_demo_df.csv', header=True, index=False)

## CBC_lab

In [98]:
query_21 = """select * from MIMIC4_Hb_first"""
query_22 = """select * from MIMIC4_Hct_first"""
query_23 = """select * from MIMIC4_INR_first"""
query_24 = """select * from MIMIC4_RBC_count_first"""
query_25 = """select * from MIMIC4_WBC_count_first"""
query_26 = """select * from MIMIC4_plt_first"""
query_27 = """select * from MIMIC4_pt_first"""
query_28 = """select * from MIMIC4_ptt_first"""

In [99]:
## 쿼리 불러오기
Hb = pd.read_sql(query_21, q)
Hct = pd.read_sql(query_22, q)
INR = pd.read_sql(query_23, q)
RBC_count = pd.read_sql(query_24, q)
WBC_count = pd.read_sql(query_25, q)
plt_count = pd.read_sql(query_26, q)
pt = pd.read_sql(query_27, q)
ptt = pd.read_sql(query_28, q)

In [100]:
## id, charttime, 측정치 변수만 불러서 정렬
Hb = Hb[['subject_id','charttime','hb']].sort_values(['subject_id','charttime'])
Hct = Hct[['subject_id','charttime','hct']].sort_values(['subject_id','charttime'])
INR = INR[['subject_id','charttime','inr']].sort_values(['subject_id','charttime'])
RBC_count = RBC_count[['subject_id','charttime','rbc_count']].sort_values(['subject_id','charttime'])
WBC_count = WBC_count[['subject_id','charttime','wbc_count']].sort_values(['subject_id','charttime'])
plt_count = plt_count[['subject_id','charttime','plt_count']].sort_values(['subject_id','charttime'])
pt = pt[['subject_id','charttime','pt']].sort_values(['subject_id','charttime'])
ptt = ptt[['subject_id','charttime','ptt']].sort_values(['subject_id','charttime'])

In [101]:
lab=pd.merge(Hb, Hct, how='outer', on=['subject_id','charttime'])

In [102]:
lab=pd.merge(lab,INR, how='outer', on=['subject_id','charttime'])
lab=pd.merge(lab,RBC_count, how='outer', on=['subject_id','charttime'])
lab=pd.merge(lab,WBC_count, how='outer', on=['subject_id','charttime'])
lab=pd.merge(lab,plt_count, how='outer', on=['subject_id','charttime'])
lab=pd.merge(lab,pt, how='outer', on=['subject_id','charttime'])
lab=pd.merge(lab,ptt, how='outer', on=['subject_id','charttime'])

In [103]:
lab

,subject_id,charttime,hb,hct,inr,rbc_count,wbc_count,plt_count,pt,ptt
0,10000019,2129-05-21 21:10:00.0000000,18.0,52.6,NaN,5.17,20.3,NaN,NaN,NaN
1,10000032,2180-03-23 11:51:00.0000000,14.9,45.4,1.4,4.40,3.0,83.0,15.1,NaN
2,10000048,2126-11-22 20:45:00.0000000,13.0,38.2,NaN,4.20,9.6,194.0,NaN,NaN
3,10000084,2160-11-20 22:30:00.0000000,12.8,38.2,1.0,4.03,8.6,263.0,11.1,23.4
4,10000108,2163-09-27 18:55:00.0000000,14.5,41.7,NaN,4.92,8.5,198.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
385097,19997062,2115-10-12 22:24:00.0000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.8
385098,19997886,2186-11-13 05:35:00.0000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.2
385099,19997911,2193-09-02 09:51:00.0000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.7
385100,19998591,2184-12-04 12:20:00.0000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.2


* case group

In [104]:
case_df=case_list[['subject_id','hadm_id','stay_id','intime','outtime']]

In [105]:
case_lab=pd.merge(case_df, lab, how='left', on=['subject_id'])

In [106]:
case_lab['charttime'] = pd.to_datetime(case_lab['charttime'])

In [107]:
#case_lab = case_lab[case_lab['intime']<=case_lab['charttime']]
#case_lab = case_lab[case_lab['charttime']<=case_lab['outtime']]

In [108]:
case_lab['timedelta'] = case_lab['charttime'] - case_lab['intime']
case_lab['timedelta_min'] = (case_lab['charttime'] - case_lab['intime'])/pd.Timedelta('1 minutes')

In [109]:
## lab 결과 intime과 charttime사이에 평균이 -6494017분.??...;;;
sum(case_lab['timedelta_min'])/len(case_lab['timedelta_min'])

-694017.4201173678

In [110]:
## 중환자실 입실 이후 -24시간(1440분) 에서 60분 이내 측정된 초기값만...
case_lab = case_lab[-1440<=case_lab['timedelta_min']]
case_lab = case_lab[case_lab['timedelta_min']<=60]

In [111]:
## groupby 위한 id 열 복제
case_lab['subject_id_index'] = case_lab['subject_id']

In [112]:
case_lab_groupby=case_lab.groupby('subject_id_index').fillna(method='ffill').fillna(method='bfill')

In [113]:
case_lab_df = case_lab_groupby.sort_values(['subject_id','intime']).drop_duplicates(['subject_id'], keep='first')

In [114]:
case_lab_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2683 entries, 10 to 14191
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype          
---  ------         --------------  -----          
 0   subject_id     2683 non-null   int64          
 1   hadm_id        2683 non-null   int64          
 2   stay_id        2683 non-null   int64          
 3   intime         2683 non-null   datetime64[ns] 
 4   outtime        2683 non-null   datetime64[ns] 
 5   charttime      2683 non-null   datetime64[ns] 
 6   hb             2683 non-null   float64        
 7   hct            2683 non-null   float64        
 8   inr            2683 non-null   float64        
 9   rbc_count      2683 non-null   float64        
 10  wbc_count      2682 non-null   float64        
 11  plt_count      2683 non-null   float64        
 12  pt             2683 non-null   float64        
 13  ptt            2683 non-null   float64        
 14  timedelta      2683 non-null   timedelta64[ns]
 15  ti

In [115]:
case_lab_df.describe()

,subject_id,hadm_id,stay_id,hb,hct,inr,rbc_count,wbc_count,plt_count,pt,ptt,timedelta,timedelta_min
count,2.683000e+03,2.683000e+03,2.683000e+03,2683.000000,2683.000000,2683.000000,2683.000000,2682.000000,2683.000000,2683.000000,2683.000000,2683,2683.000000
mean,1.496658e+07,2.493293e+07,3.495851e+07,9.718077,29.457138,1.586359,3.219109,14.365406,215.132315,17.249497,35.870257,-1 days +19:02:09.677226985,-297.838713
std,2.868345e+06,2.886125e+06,2.893478e+06,2.580813,7.410704,1.283754,0.853531,22.625830,125.425379,12.581275,22.758848,0 days 05:32:56.879322009,332.947989
min,1.000605e+07,2.000700e+07,3.000551e+07,2.200000,6.500000,0.800000,0.450000,0.000000,5.000000,8.400000,17.400000,-1 days +00:03:22,-1436.633333
25%,1.252278e+07,2.239852e+07,3.236469e+07,7.800000,24.000000,1.100000,2.580000,7.600000,134.000000,12.200000,25.700000,-1 days +18:05:00,-355.000000
50%,1.500070e+07,2.486968e+07,3.492158e+07,9.600000,28.900000,1.200000,3.170000,11.200000,200.000000,13.700000,29.400000,-1 days +20:40:00,-200.000000
75%,1.739941e+07,2.743010e+07,3.746652e+07,11.600000,34.800000,1.600000,3.830000,16.200000,270.000000,17.200000,35.000000,-1 days +22:30:00,-90.000000
max,1.999231e+07,2.999840e+07,3.999377e+07,19.200000,61.200000,24.600000,6.560000,479.200000,1164.000000,150.000000,150.000000,0 days 00:58:48,58.800000


In [116]:
## 99백분위수 극단값 확인
print(case_lab_df['hb'].quantile(0.99))
print(case_lab_df['hct'].quantile(0.99))
print(case_lab_df['inr'].quantile(0.99))
print(case_lab_df['rbc_count'].quantile(0.99))
print(case_lab_df['wbc_count'].quantile(0.99))
print(case_lab_df['plt_count'].quantile(0.99))
print(case_lab_df['pt'].quantile(0.99))
print(case_lab_df['ptt'].quantile(0.99))

15.3
47.01799999999999
6.717999999999983
5.201799999999999
62.08300000000031
615.5399999999995
69.33599999999996
150.0


In [117]:
## 이상치 99백분위수 극단값으로 대체
case_lab_df['hb']=case_lab_df['hb'].apply(lambda x:x if x>=0 and x<=15.3 else 15.3)
case_lab_df['hct']=case_lab_df['hct'].apply(lambda x:x if x>=0 and x<=47.0 else 47.0)
case_lab_df['inr']=case_lab_df['inr'].apply(lambda x:x if x>=0 and x<=6.7 else 6.7)
case_lab_df['rbc_count']=case_lab_df['rbc_count'].apply(lambda x:x if x>=0 and x<=5.2 else 5.2)
case_lab_df['wbc_count']=case_lab_df['wbc_count'].apply(lambda x:x if x>=0 and x<=62.0 else 62.0)
case_lab_df['plt_count']=case_lab_df['plt_count'].apply(lambda x:x if x>=0 and x<=615.5 else 615.5)
case_lab_df['pt']=case_lab_df['pt'].apply(lambda x:x if x>=0 and x<=69.3 else 69.3)
case_lab_df['ptt']=case_lab_df['ptt'].apply(lambda x:x if x>=0 and x<=150 else 150.0)

In [118]:
## CSV내보내기
case_lab_df.to_csv('./case_lab.csv', header=True, index=False)

* control_group

In [119]:
control_df=control_list[['subject_id','hadm_id','stay_id','intime','outtime']]

In [120]:
control_lab=pd.merge(control_df, lab, how='left', on=['subject_id'])

In [121]:
control_lab['charttime'] = pd.to_datetime(control_lab['charttime'])

In [122]:
control_lab['timedelta'] = control_lab['charttime'] - control_lab['intime']
control_lab['timedelta_min'] = (control_lab['charttime'] - control_lab['intime'])/pd.Timedelta('1 minutes')

In [123]:
## lab 결과 intime과 charttime사이에 평균이 -694017분.??...;;;
sum(control_lab['timedelta_min'])/len(control_lab['timedelta_min'])

nan

In [124]:
## 중환자실 입실 이후 -24시간(1440분) 에서 60분 이내 측정된 초기값만...
control_lab = control_lab[-1440<=control_lab['timedelta_min']]
control_lab = control_lab[control_lab['timedelta_min']<=60]

In [125]:
## groupby 위한 id 열 복제
control_lab['subject_id_index'] = control_lab['subject_id']

In [126]:
control_lab_groupby=control_lab.groupby('subject_id_index').fillna(method='ffill').fillna(method='bfill')

In [127]:
control_lab_df = control_lab_groupby.sort_values(['subject_id','intime']).drop_duplicates(['subject_id'], keep='first')

In [128]:
control_lab_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19447 entries, 11 to 62210
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype          
---  ------         --------------  -----          
 0   subject_id     19447 non-null  int64          
 1   hadm_id        19447 non-null  int64          
 2   stay_id        19447 non-null  int64          
 3   intime         19447 non-null  datetime64[ns] 
 4   outtime        19447 non-null  datetime64[ns] 
 5   charttime      19447 non-null  datetime64[ns] 
 6   hb             19447 non-null  float64        
 7   hct            19447 non-null  float64        
 8   inr            19447 non-null  float64        
 9   rbc_count      19447 non-null  float64        
 10  wbc_count      19447 non-null  float64        
 11  plt_count      19447 non-null  float64        
 12  pt             19447 non-null  float64        
 13  ptt            19447 non-null  float64        
 14  timedelta      19447 non-null  timedelta64[ns]
 15  t

In [129]:
control_lab_df.describe()

,subject_id,hadm_id,stay_id,hb,hct,inr,rbc_count,wbc_count,plt_count,pt,ptt,timedelta,timedelta_min
count,1.944700e+04,1.944700e+04,1.944700e+04,19447.000000,19447.000000,19447.000000,19447.000000,19447.000000,19447.000000,19447.000000,19447.000000,19447,19447.000000
mean,1.497462e+07,2.496798e+07,3.498883e+07,12.692045,38.155515,155.597505,4.209727,12.340347,285.396514,168.875518,33.585278,-1 days +19:11:58.486347509,-288.025228
std,2.884868e+06,2.883363e+06,2.874253e+06,2.146148,6.068479,12419.696234,0.723588,11.519471,7169.934218,12419.534094,20.264821,0 days 04:53:42.404038748,293.706734
min,1.000222e+07,2.000009e+07,3.000019e+07,0.000000,0.000000,0.700000,0.000000,0.000000,5.000000,8.200000,15.300000,-1 days +00:00:20,-1439.666667
25%,1.245610e+07,2.249511e+07,3.251887e+07,11.300000,34.300000,1.000000,3.740000,7.500000,173.000000,11.500000,25.500000,-1 days +18:11:00,-349.000000
50%,1.497935e+07,2.491454e+07,3.498830e+07,12.800000,38.400000,1.100000,4.240000,10.400000,222.000000,12.600000,28.500000,-1 days +20:29:00,-211.000000
75%,1.748132e+07,2.748753e+07,3.744607e+07,14.200000,42.200000,1.300000,4.690000,14.700000,280.000000,14.300000,32.700000,-1 days +22:00:00,-120.000000
max,1.999999e+07,2.999983e+07,3.999866e+07,21.400000,65.400000,999999.000000,7.750000,385.900000,999999.000000,999999.000000,150.000000,0 days 01:00:00,60.000000


In [130]:
## 99백분위수 극단값 확인
print(control_lab_df['hb'].quantile(0.99))
print(control_lab_df['hct'].quantile(0.99))
print(control_lab_df['inr'].quantile(0.99))
print(control_lab_df['rbc_count'].quantile(0.99))
print(control_lab_df['wbc_count'].quantile(0.99))
print(control_lab_df['plt_count'].quantile(0.99))
print(control_lab_df['pt'].quantile(0.99))
print(control_lab_df['ptt'].quantile(0.99))

17.3
52.2
4.8
5.89
41.0
561.5400000000009
49.454000000000086
150.0


In [131]:
## 이상치 99백분위수 극단값으로 대체
control_lab_df['hb']=control_lab_df['hb'].apply(lambda x:x if x>=0 and x<=17.3 else 17.3)
control_lab_df['hct']=control_lab_df['hct'].apply(lambda x:x if x>=0 and x<=52.2 else 52.2)
control_lab_df['inr']=control_lab_df['inr'].apply(lambda x:x if x>=0 and x<=4.8 else 4.8)
control_lab_df['rbc_count']=control_lab_df['rbc_count'].apply(lambda x:x if x>=0 and x<=5.9 else 5.9)
control_lab_df['wbc_count']=control_lab_df['wbc_count'].apply(lambda x:x if x>=0 and x<=41.0 else 41.0)
control_lab_df['plt_count']=control_lab_df['plt_count'].apply(lambda x:x if x>=0 and x<=561.5 else 561.5)
control_lab_df['pt']=control_lab_df['pt'].apply(lambda x:x if x>=0 and x<=49.5 else 49.5)
control_lab_df['ptt']=control_lab_df['ptt'].apply(lambda x:x if x>=0 and x<=150.0 else 150.0)

In [132]:
control_lab_df

,subject_id,hadm_id,stay_id,intime,outtime,charttime,hb,hct,inr,rbc_count,wbc_count,plt_count,pt,ptt,timedelta,timedelta_min
11,10002223,22494570,39908879,2158-01-15 08:01:49,2158-01-16 15:19:24,2158-01-15 06:55:00,12.9,38.6,1.2,4.31,19.6,186.0,14.3,23.1,-1 days +22:53:11,-66.816667
13,10002443,21329021,35217462,2183-10-18 00:47:00,2183-10-20 18:48:03,2183-10-17 22:57:00,14.8,45.0,1.3,4.84,19.6,215.0,13.7,27.2,-1 days +22:10:00,-110.000000
16,10002618,22192064,30044988,2173-12-03 09:30:00,2173-12-03 18:17:11,2173-12-03 08:00:00,14.8,45.0,1.0,4.84,19.6,215.0,11.1,25.2,-1 days +22:30:00,-90.000000
27,10004235,24181354,30276431,2196-02-24 17:07:00,2196-02-29 15:58:02,2196-02-24 12:40:00,11.7,35.0,1.3,4.21,24.0,31.0,14.6,35.9,-1 days +19:33:00,-267.000000
30,10004606,29242151,32639376,2159-02-20 16:28:04,2159-02-25 20:09:14,2159-02-20 12:00:00,16.2,48.4,0.9,5.35,15.7,212.0,10.2,29.7,-1 days +19:31:56,-268.066667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62187,19997843,20277361,30733813,2120-11-18 02:39:00,2120-11-20 23:02:24,2120-11-17 21:40:00,14.8,46.1,1.3,4.49,12.4,146.0,14.6,30.3,-1 days +19:01:00,-299.000000
62198,19998843,24842066,37372228,2187-02-05 10:12:00,2187-02-08 18:19:39,2187-02-05 08:50:00,10.5,30.8,1.2,3.35,11.7,151.0,12.9,26.7,-1 days +22:38:00,-82.000000
62201,19999068,21606769,31096823,2161-08-24 05:26:00,2161-08-30 23:48:04,2161-08-24 03:45:00,12.3,36.8,1.1,3.67,6.1,109.0,12.2,27.3,-1 days +22:19:00,-101.000000
62207,19999625,25304202,38646795,2139-10-10 19:18:00,2139-10-11 18:21:28,2139-10-10 16:45:00,14.1,42.2,1.1,4.07,0.0,136.0,13.1,22.8,-1 days +21:27:00,-153.000000


In [133]:
## CSV내보내기
control_lab_df.to_csv('./control_lab_df.csv', header=True, index=False)